### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score

### Load Data

In [27]:
df = pd.read_csv('./datasets/train.csv')
df_test = pd.read_csv('./datasets/test.csv')
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


### Data Cleaning Initial Check

In [3]:
# Checking for null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2051 non-null   int64  
 1   PID              2051 non-null   int64  
 2   MS SubClass      2051 non-null   int64  
 3   MS Zoning        2051 non-null   object 
 4   Lot Frontage     1721 non-null   float64
 5   Lot Area         2051 non-null   int64  
 6   Street           2051 non-null   object 
 7   Alley            140 non-null    object 
 8   Lot Shape        2051 non-null   object 
 9   Land Contour     2051 non-null   object 
 10  Utilities        2051 non-null   object 
 11  Lot Config       2051 non-null   object 
 12  Land Slope       2051 non-null   object 
 13  Neighborhood     2051 non-null   object 
 14  Condition 1      2051 non-null   object 
 15  Condition 2      2051 non-null   object 
 16  Bldg Type        2051 non-null   object 
 17  House Style   

In [4]:
# These features are contain a lot of loss of info

df.drop(columns=['Alley','Pool QC','Misc Feature', 'Fence'])

# The following features should be considered for
# imputation; next round check
imputable_cols = ['Lot Frontage', 'Bsmt Qual', 'Bsmt Cond', 
                  'Bsmt Exposure', 'BsmtFin Type 1','Garage Yr Blt',
                  'Garage Finish', 'Garage Qual', 'Garage Cond']

In [5]:
# Drop rows containing nan's
print(f'Rows before droping nan\'s: {df.shape[0]}')
temp = df.dropna()
print(f'Rows after droping nan\'s: {temp.shape[0]}')

Rows before droping nan's: 2051
Rows after droping nan's: 0


### 1st model iteration:

In [6]:
# droping columns based on missing nans
print(f'Cols before droping nan\'s: {df.shape[1]}')
temp = df.dropna(axis='columns')
print(f'Cols after droping nan\'s: {temp.shape[1]}')

Cols before droping nan's: 81
Cols after droping nan's: 55


In [7]:
df = temp

In [8]:
df.corr()['SalePrice'].sort_values()

PID               -0.255052
Enclosed Porch    -0.135656
Kitchen AbvGr     -0.125444
Overall Cond      -0.097019
MS SubClass       -0.087335
Id                -0.051398
Low Qual Fin SF   -0.041594
Yr Sold           -0.015203
Misc Val          -0.007375
Pool Area          0.023106
Mo Sold            0.032735
3Ssn Porch         0.048732
Screen Porch       0.134581
Bedroom AbvGr      0.137067
2nd Flr SF         0.248452
Half Bath          0.283001
Lot Area           0.296566
Wood Deck SF       0.326490
Open Porch SF      0.333476
Fireplaces         0.471093
TotRms AbvGrd      0.504014
Full Bath          0.537969
Year Remod/Add     0.550370
Year Built         0.571849
1st Flr SF         0.618486
Gr Liv Area        0.697038
Overall Qual       0.800207
SalePrice          1.000000
Name: SalePrice, dtype: float64

In [9]:
# ** Note ** : list contains predicted label of salePrice  
features_high_corr = ['Fireplaces', 'TotRms AbvGrd', 'Full Bath', 'Year Remod/Add', 
                      'Year Built', '1st Flr SF', 'Gr Liv Area', 'Overall Qual', 'SalePrice']

In [10]:
df_first_iter = df[features_high_corr]

In [11]:
df[features_high_corr].dtypes

Fireplaces        int64
TotRms AbvGrd     int64
Full Bath         int64
Year Remod/Add    int64
Year Built        int64
1st Flr SF        int64
Gr Liv Area       int64
Overall Qual      int64
SalePrice         int64
dtype: object

In [12]:
set(df['Full Bath'])

{0, 1, 2, 3, 4}

In [13]:
df_first_iter = pd.get_dummies(df_first_iter, columns=['Full Bath'], prefix='FullBath')

In [14]:
set(df['Overall Qual'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [15]:
df_first_iter = pd.get_dummies(df_first_iter, columns=['Overall Qual'], prefix='OverallQual')

In [16]:
set(df['Fireplaces'])

{0, 1, 2, 3, 4}

In [17]:
df_first_iter = pd.get_dummies(df_first_iter, columns=['Fireplaces'])

In [18]:
set(df['TotRms AbvGrd'])

{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [19]:
df_first_iter = pd.get_dummies(df_first_iter, columns=['TotRms AbvGrd'], prefix='TotRms_AbvGrd')

### Model Prep

In [20]:
final_features = list(df_first_iter.columns)
final_features.remove('SalePrice')

X = df_first_iter[final_features]
y = df_first_iter['SalePrice']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=30)

In [22]:
lr = LinearRegression()

In [23]:
lr_scores = cross_val_score(lr, X_train, y_train, cv=3)
lr_scores.mean()

0.863003312624074

In [24]:
lr.fit(X_train, y_train)

LinearRegression()

In [25]:
lr.score(X_train, y_train)

0.8771337763873388

In [26]:
lr.score(X_test, y_test)

0.6274430936904717

In [39]:
X.head()

,Year Remod/Add,Year Built,1st Flr SF,Gr Liv Area,FullBath_0,FullBath_1,FullBath_2,FullBath_3,FullBath_4,OverallQual_1,...,TotRms_AbvGrd_6,TotRms_AbvGrd_7,TotRms_AbvGrd_8,TotRms_AbvGrd_9,TotRms_AbvGrd_10,TotRms_AbvGrd_11,TotRms_AbvGrd_12,TotRms_AbvGrd_13,TotRms_AbvGrd_14,TotRms_AbvGrd_15
0,2005,1976,725,1479,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1997,1996,913,2122,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2007,1953,1057,1057,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2007,2006,744,1444,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,1993,1900,831,1445,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


### Now we have a model object, and we can run our df_test data through it

In [58]:
def data_cleaning(data_frame_test, data_frame_train):
    data_frame_test.drop(columns=['Alley','Pool QC','Misc Feature', 'Fence'])
    data_frame_test.dropna(axis='columns')
    
    # features with high corr 
    features_high_corr = ['Fireplaces', 'TotRms AbvGrd', 'Full Bath', 'Year Remod/Add', 
                      'Year Built', '1st Flr SF', 'Gr Liv Area', 'Overall Qual']
    df_first_iter = data_frame_test[features_high_corr]
    
    #dummies 
    df_first_iter = pd.get_dummies(df_first_iter, columns=['Full Bath'], prefix='FullBath')
    df_first_iter = pd.get_dummies(df_first_iter, columns=['Overall Qual'], prefix='OverallQual')
    df_first_iter = pd.get_dummies(df_first_iter, columns=['Fireplaces'])
    df_first_iter = pd.get_dummies(df_first_iter, columns=['TotRms AbvGrd'], prefix='TotRms_AbvGrd')
    
    final_features = list(df_first_iter.columns)

    X = df_first_iter[final_features]
    
    a1, a2 = data_frame_train.align(X, join='outer', axis=1)
    
    a2.drop(columns=['SalePrice'], inplace=True)
    return a1, a2

In [60]:
a1, a2 = data_cleaning(df_test, df_first_iter)
print(a2.shape)
print(lr.predict(a2).shape)
# df_test['y_hat'] = lr.predict(X_TEST)

a2.columns

(878, 38)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [49]:
a1.columns

Index(['1st Flr SF', 'Fireplaces_0', 'Fireplaces_1', 'Fireplaces_2',
       'Fireplaces_3', 'Fireplaces_4', 'FullBath_0', 'FullBath_1',
       'FullBath_2', 'FullBath_3', 'FullBath_4', 'Gr Liv Area',
       'OverallQual_1', 'OverallQual_10', 'OverallQual_2', 'OverallQual_3',
       'OverallQual_4', 'OverallQual_5', 'OverallQual_6', 'OverallQual_7',
       'OverallQual_8', 'OverallQual_9', 'SalePrice', 'TotRms_AbvGrd_10',
       'TotRms_AbvGrd_11', 'TotRms_AbvGrd_12', 'TotRms_AbvGrd_13',
       'TotRms_AbvGrd_14', 'TotRms_AbvGrd_15', 'TotRms_AbvGrd_2',
       'TotRms_AbvGrd_3', 'TotRms_AbvGrd_4', 'TotRms_AbvGrd_5',
       'TotRms_AbvGrd_6', 'TotRms_AbvGrd_7', 'TotRms_AbvGrd_8',
       'TotRms_AbvGrd_9', 'Year Built', 'Year Remod/Add'],
      dtype='object')